In [15]:
import numpy as np
import pandas as pd
import os

pathdata = '/mydata/watres/quentin/code/FLOW/data/GISID2hourly_data_withPET/'
all_GISID = np.array([el[:-4] for el in os.listdir(pathdata)])
all_GISID = np.array([el for el in all_GISID if not('ipynb_checkpo' in el)])

GISID = all_GISID[0]
df = pd.read_csv('/mydata/watres/quentin/code/FLOW/data/GISID2hourly_data_withPET/{0}.csv'.format(GISID), sep=',')
df

,discharge,precip,t,datetime,tmin,tmax,tabs,pet
0,0.219167,0.046603,2005.000000,2005-01-01 00:00:00,1.658938,7.095829,3.673320,0.435224
1,0.210833,0.000033,2005.000114,2005-01-01 01:00:00,1.658938,7.095829,3.673320,0.435224
2,0.206833,0.056963,2005.000228,2005-01-01 02:00:00,1.658938,7.095829,3.673320,0.435224
3,0.208500,0.034366,2005.000342,2005-01-01 03:00:00,1.658938,7.095829,3.673320,0.435224
4,0.210500,0.011202,2005.000457,2005-01-01 04:00:00,1.658938,7.095829,3.673320,0.435224
...,...,...,...,...,...,...,...,...
140228,0.202000,0.000000,2020.996812,2020-12-30 20:00:00,-0.239202,3.591478,1.395561,0.324806
140229,0.201500,0.000000,2020.996926,2020-12-30 21:00:00,-0.239202,3.591478,1.395561,0.324806
140230,0.201000,0.000000,2020.997040,2020-12-30 22:00:00,-0.239202,3.591478,1.395561,0.324806
140231,0.200167,0.000000,2020.997154,2020-12-30 23:00:00,-0.239202,3.591478,1.395561,0.324806


In [14]:
for GISID in all_GISID:
    df = pd.read_csv('/mydata/watres/quentin/code/FLOW/data/GISID2hourly_data_withPET/{0}.csv'.format(GISID), sep=',')
    df = df.rename(columns={"discharge": "q"})
    df = df.rename(columns={"t": "timeyear"})
    df = df.rename(columns={"datetime": "date"})
    df = df.rename(columns={"precip": "p"})
    df = df.fillna(0)
    df = df.loc[df['timeyear']>2014]
    df = df.reset_index()
    import os
    directory = '/mydata/watres/quentin/code/FLOW/hourly_analysis/RES_GAMCR/real_data/{0}/'.format(GISID)
    if not os.path.exists(directory):
        os.makedirs(directory)
    df.to_csv(directory+'data_{0}.txt'.format(GISID), index=False)

In [29]:
for GISID in all_GISID:
    try:
        directory = '/mydata/watres/quentin/code/FLOW/hourly_analysis/RES_GAMCR/real_data/{0}/'.format(GISID)
        df = pd.read_csv(directory+'data_{0}.txt'.format(GISID))
        df['date'] = pd.to_datetime(df['date'])
        
        # Compute the difference between consecutive dates
        date_diffs = df['date'].diff()
        
        # Find the maximum difference
        max_diff = date_diffs.max()
        print(GISID, max_diff)
    except:
        pass

325 0 days 01:00:00
167 0 days 01:00:00
69 0 days 01:00:00
98 0 days 01:00:00
187 0 days 01:00:00
164 0 days 01:00:00
252 0 days 01:00:00
14 0 days 01:00:00
156 0 days 01:00:00
27 0 days 01:00:00
192 0 days 01:00:00
99 0 days 01:00:00
191 0 days 01:00:00
173 0 days 01:00:00
113 0 days 01:00:00
233 0 days 01:00:00
225 0 days 01:00:00
101 0 days 01:00:00
123 0 days 01:00:00
116 0 days 01:00:00
329 0 days 01:00:00
95 0 days 01:00:00
168 0 days 01:00:00
129 0 days 01:00:00
49 0 days 01:00:00
389 0 days 01:00:00
182 0 days 01:00:00
219 0 days 01:00:00
80 0 days 01:00:00
1 0 days 01:00:00
170 0 days 01:00:00
162 0 days 01:00:00
198 0 days 01:00:00
402 0 days 01:00:00
331 0 days 01:00:00
160 0 days 01:00:00
38 0 days 01:00:00
320 0 days 01:00:00
46 0 days 01:00:00
24 0 days 01:00:00
72 0 days 01:00:00
143 0 days 01:00:00
253 0 days 01:00:00
254 0 days 01:00:00
401 0 days 01:00:00
261 0 days 01:00:00
258 0 days 01:00:00
200 0 days 01:00:00
251 0 days 01:00:00
193 0 days 01:00:00
50 0 days 01:0

In [28]:
max_diff

Timedelta('0 days 01:00:00')